In [1]:
import pandas as pd

In [2]:
# df['Address-B'].unique()

In [3]:
# df[df['Address-B'] == '192.168.1.66']

### Gloabl Vars

In [4]:
CATEGORY_DICT = {"Slow_HTTP_GET_64" : "192.168.1.64",
                "Slow_Read_65" : "192.168.1.65",
                "Slow_Post_66" : "192.168.1.66",
                "GoldenEye_67" : "192.168.1.67",
                "Hulk_68" : "192.168.1.68",
                "Benign_high_latency_2_83" : "192.168.2.83",
                "Benign_low_latency_2_183" : "192.168.2.183",
                "Reciever_Address" : "192.168.10.124"}

full_conversation_cols = ["Address-A", "Port-A", "Address-B", "Port-B", "Packets", "Bytes", "StreamID", "TotalPackets", 
                          "PerctageFiltered", "Packets_A-to-B(TX_pkts)", "Bytes_A-to-B(TX_bytes)", "Packets_B-to-A(RX_pkts)",
                          "Bytes_B-to-A(RX_bytes)", "Relative_Start", "Duration", "Bits/s A-to-B (TX_bits/s)",
                          "Bits/s B-to-A (RX_bits/s)"]

FLAG_DICT = {"Not set":0, "Set":1}

FULL_CONVERSATION_FILE = 'DDOS_Conversation_list.csv'

SLOW_HTTP_GET_PATH = "Slow_HTTP_GET_64"
SLOW_READ_PATH = "Slow_Read_65"
SLOW_POST_PATH = "Slow_Post_66"
GOLDENEYE_PATH = "GoldenEye_67"
HULK_PATH = "Hulk_68"
BENIGN_HIGH_LATENCY_PATH = "Benign_high_latency_2_83"
BENIGN_LOW_LATENCY_PATH = "Benign_low_latency_2_183"

individual_packets_suffix = "_individual_packets"

### Steps to do in wireshark

1. Filter on all ip addresses except the *Reciever_Address* in `CATEGORY_DICT`
2. Select columns : Stream Index, Header Length, TCP Segment Length, Length, Bytes in Flight, All flags (PSH, RST, FIN, SYN, ACK) and add them to the display panel
3. Order by Stream Index and export packets in display panel as a csv file
4. Click on Statistics -> Conversations. Sort by stream index which will be useful later when merging is used in pandas. Export the conversations as CSV

### Data Preprocessing

In [16]:
def get_full_conversation_df(need_cols=False):
    if need_cols:
        df = pd.read_csv(FULL_CONVERSATION_FILE, names=full_conversation_cols, header=None)
    else:
        df = pd.read_csv(FULL_CONVERSATION_FILE)
    # Replace NaN in Bits/s A-to-B (TX_bits/s) and Bits/s B-to-A (RX_bits/s) with 0
    df['Bits/s A-to-B (TX_bits/s)'].fillna(0, inplace=True)
    df['Bits/s B-to-A (RX_bits/s)'].fillna(0, inplace=True)
    return df

In [17]:
def get_ddos_category_packets_df(category_path):
    df_category = pd.read_csv(category_path + '/' + category_path + individual_packets_suffix + '.csv')
    return df_category

In [18]:
def remove_nan_Bytes_In_Flight(df_packets):
    df_packets['Bytes In Flight'] = df_packets['Bytes In Flight'].fillna(0)
    return df_packets

In [19]:
def map_flags_to_numeric(df_packets):
    df_packets['Push'] = df_packets['Push'].map(FLAG_DICT)
    df_packets['Syn'] = df_packets['Syn'].map(FLAG_DICT)
    df_packets['Acknowledgment'] = df_packets['Acknowledgment'].map(FLAG_DICT)
    df_packets['Fin'] = df_packets['Fin'].map(FLAG_DICT)
    df_packets['Reset'] = df_packets['Reset'].map(FLAG_DICT)
    return df_packets

In [42]:
def stream_index_groupby(category, df_packets, df):
    
    # Filter by up address on full conversation df
    df_conversation = df[df['Address-A'] == CATEGORY_DICT[category]].copy()
    
    # Group by stream index and sum Push, Syn, Acknowledgment, Fin, Reset packets
    df_flag_packet_count = df_packets[['Stream index', 'Syn', 'Acknowledgment', 'Fin', 'Reset', 'Push']].groupby('Stream index').sum()
    
    # Merge 'Syn', 'Acknowledgment', 'Fin', 'Reset', 'Push(PSH)' counts in conversation df
    df_conversation = df_conversation.merge(df_flag_packet_count, left_on='StreamID', right_index=True)
    
    # Group by stream index and take mean of 'Length', 'TCP Segment Len', 'Header Length', 'Bytes In Flight'
    df_attr_len_mean = df_packets[['Stream index', 'Length', 'TCP Segment Len', 'Header Length', 'Bytes In Flight']].groupby('Stream index').mean()
    
    # Merge mean of 'Length', 'TCP Segment Len', 'Header Length', 'Bytes In Flight' in conversation df
    df_conversation = df_conversation.merge(df_attr_len_mean, left_on='StreamID', right_index=True)
    
    # Append Category columns to df and sum Flags into one column
    df_conversation['flags'] = df_conversation[['Syn', 'Acknowledgment', 'Fin', 'Reset', 'Push']].sum(axis=1)
    df_conversation['CATEGORY'] = " ".join(category.split('_')[:-1])
    
    
    return df_conversation

In [43]:
def preprocess_data(category):
    # Get the full conversation DataFrame
    df_full_conversations = get_full_conversation_df()
    
    # Get DataFrame conatining individual packets for the category
    df_cat_ind_pkts = get_ddos_category_packets_df(category)
    
    # Remove NaN in Bytes in Flight Column
    df_cat_ind_pkts = remove_nan_Bytes_In_Flight(df_cat_ind_pkts)
    
    # Map Flags 'Syn', 'Acknowledgment', 'Fin', 'Reset', 'Push(PSH)' as - Not Set : 0, Set : 1
    df_cat_ind_pkts = map_flags_to_numeric(df_cat_ind_pkts)
    
    # Apply Groupy by on Stream Index and aggregate Flags by Sum and 
    # 'Length', 'TCP Segment Len', 'Header Length', 'Bytes In Flight' by mean.
    # After aggregate apply mask of ip address using CATEGORY_DICT on full conversations DataFrame
    # and merge it with individual packets Dataframe on Stream Index
    df_category_final = stream_index_groupby(category, df_cat_ind_pkts, df_full_conversations)
    
    return df_category_final
    

In [44]:
# df_slow_http_get_packets = get_ddos_category_packets_df(SLOW_HTTP_GET_PATH)
# df_slow_read_packets = get_ddos_category_packets_df(SLOW_READ_PATH)
# df_slow_post_packets = get_ddos_category_packets_df(SLOW_POST_PATH)
# df_goldeneye_packets = get_ddos_category_packets_df(GOLDENEYE_PATH)
# df_hulk_packets = get_ddos_category_packets_df(HULK_PATH)
# df_benign_high_latency_packets = get_ddos_category_packets_df(BENIGN_HIGH_LATENCY_PATH)
# df_benign_low_latency_packets = get_ddos_category_packets_df(BENIGN_LOW_LATENCY_PATH)

In [45]:
# df = get_full_conversation_df()

In [51]:
df_slow_http_get = preprocess_data(SLOW_HTTP_GET_PATH)
df_slow_read = preprocess_data(SLOW_READ_PATH)
df_slow_post = preprocess_data(SLOW_POST_PATH)
df_goldeneye = preprocess_data(GOLDENEYE_PATH)
df_hulk = preprocess_data(HULK_PATH)
# df_benign_high_latency = preprocess_data(BENIGN_HIGH_LATENCY_PATH)
# df_benign_low_latency = preprocess_data(BENIGN_LOW_LATENCY_PATH)

In [52]:
len(df_slow_http_get) + len(df_slow_read) + len(df_slow_post) + len(df_goldeneye) + len(df_hulk)

31457

In [53]:
df_ddos_final = pd.concat([df_slow_http_get, df_slow_read, df_slow_post, df_goldeneye, df_hulk])

In [54]:
len(df_ddos_final)

31457

In [55]:
df_ddos_final.to_csv('DDOS_Traffic/ddos_traffic_model_ready.csv', index=False)

In [56]:
new_df = pd.read_csv('DDOS_Traffic/ddos_traffic_model_ready.csv')

In [57]:
new_df.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Acknowledgment,Fin,Reset,Push,Length,TCP Segment Len,Header Length,Bytes In Flight,flags,CATEGORY
0,192.168.1.64,34410,192.168.10.124,80,0,0,10,2,100,1,...,1,1,1,1,71.000000,11.000000,26.000000,11.000000,4,Slow HTTP GET
1,192.168.1.64,51450,192.168.10.124,80,0,0,44,24,100,13,...,17,2,6,5,542.416667,477.250000,29.666667,2166.583333,32,Slow HTTP GET
2,192.168.1.64,51452,192.168.10.124,80,0,0,45,14,100,7,...,13,2,0,4,125.857143,58.714286,33.142857,58.714286,21,Slow HTTP GET
3,192.168.1.64,51454,192.168.10.124,80,0,0,46,14,100,7,...,13,2,0,4,126.714286,59.571429,33.142857,59.571429,21,Slow HTTP GET
4,192.168.1.64,51456,192.168.10.124,80,0,0,47,14,100,7,...,13,2,0,4,124.785714,57.642857,33.142857,57.642857,21,Slow HTTP GET


In [25]:
df_ddos_final.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
10,192.168.1.64,34410,192.168.10.124,80,2,142,10,2,100,1,...,0.0,0,1,1,1,71.000000,11.000000,26.000000,11.000000,Slow HTTP GET
44,192.168.1.64,51450,192.168.10.124,80,24,13018,44,24,100,13,...,66867.0,2,17,2,6,542.416667,477.250000,29.666667,2166.583333,Slow HTTP GET
45,192.168.1.64,51452,192.168.10.124,80,14,1762,45,14,100,7,...,359.0,2,13,2,0,125.857143,58.714286,33.142857,58.714286,Slow HTTP GET
46,192.168.1.64,51454,192.168.10.124,80,14,1774,46,14,100,7,...,359.0,2,13,2,0,126.714286,59.571429,33.142857,59.571429,Slow HTTP GET
47,192.168.1.64,51456,192.168.10.124,80,14,1747,47,14,100,7,...,360.0,2,13,2,0,124.785714,57.642857,33.142857,57.642857,Slow HTTP GET


In [73]:
df_slow_http_get.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
10,192.168.1.64,34410,192.168.10.124,80,2,142,10,2,100,1,...,0.0,0,1,1,1,71.000000,11.000000,26.000000,11.000000,Slow HTTP GET
44,192.168.1.64,51450,192.168.10.124,80,24,13018,44,24,100,13,...,66867.0,2,17,2,6,542.416667,477.250000,29.666667,2166.583333,Slow HTTP GET
45,192.168.1.64,51452,192.168.10.124,80,14,1762,45,14,100,7,...,359.0,2,13,2,0,125.857143,58.714286,33.142857,58.714286,Slow HTTP GET
46,192.168.1.64,51454,192.168.10.124,80,14,1774,46,14,100,7,...,359.0,2,13,2,0,126.714286,59.571429,33.142857,59.571429,Slow HTTP GET
47,192.168.1.64,51456,192.168.10.124,80,14,1747,47,14,100,7,...,360.0,2,13,2,0,124.785714,57.642857,33.142857,57.642857,Slow HTTP GET


In [74]:
df_slow_read.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
10760,192.168.1.65,53716,192.168.10.124,80,0,0,10760,24,0,0,...,0.0,2,16,2,7,542.166667,477.250000,29.166667,2166.583333,Slow Read
10761,192.168.1.65,53718,192.168.10.124,80,0,0,10761,39,0,0,...,0.0,2,38,0,1,147.307692,80.897436,32.410256,109.615385,Slow Read
10762,192.168.1.65,53720,192.168.10.124,80,0,0,10762,39,0,0,...,0.0,2,38,0,1,147.307692,80.897436,32.410256,109.615385,Slow Read
10763,192.168.1.65,53722,192.168.10.124,80,0,0,10763,39,0,0,...,0.0,2,38,0,1,147.307692,80.897436,32.410256,109.615385,Slow Read
10764,192.168.1.65,53724,192.168.10.124,80,0,0,10764,39,0,0,...,0.0,2,38,0,1,147.307692,80.897436,32.410256,109.615385,Slow Read


In [75]:
df_slow_post.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
11892,192.168.1.66,51420,192.168.10.124,80,0,0,11892,24,0,0,...,0.0,2,16,2,7,540.333333,475.416667,29.166667,2164.75,Slow Post
11893,192.168.1.66,51422,192.168.10.124,80,0,0,11893,10,0,0,...,0.0,2,9,2,0,156.000000,88.400000,33.600000,88.40,Slow Post
11894,192.168.1.66,51424,192.168.10.124,80,0,0,11894,10,0,0,...,0.0,2,9,2,0,156.000000,88.400000,33.600000,88.40,Slow Post
11895,192.168.1.66,51426,192.168.10.124,80,0,0,11895,10,0,0,...,0.0,2,9,2,0,156.000000,88.400000,33.600000,88.40,Slow Post
11896,192.168.1.66,51428,192.168.10.124,80,0,0,11896,10,0,0,...,0.0,2,9,2,0,156.000000,88.400000,33.600000,88.40,Slow Post


In [76]:
df_goldeneye.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
17997,192.168.1.67,46622,192.168.10.124,80,0,0,17997,28,0,0,...,0.0,2,27,3,1,483.607143,416.178571,33.428571,1864.178571,GoldenEye
17998,192.168.1.67,46624,192.168.10.124,80,0,0,17998,28,0,0,...,0.0,2,27,2,1,521.107143,453.678571,33.428571,2263.678571,GoldenEye
17999,192.168.1.67,46626,192.168.10.124,80,0,0,17999,28,0,0,...,0.0,2,27,2,1,518.142857,450.714286,33.428571,2260.714286,GoldenEye
18000,192.168.1.67,46628,192.168.10.124,80,0,0,18000,28,0,0,...,0.0,2,27,2,1,522.928571,455.500000,33.428571,2265.500000,GoldenEye
18001,192.168.1.67,46630,192.168.10.124,80,0,0,18001,16,0,0,...,0.0,2,15,2,1,305.500000,237.750000,33.750000,509.250000,GoldenEye


In [77]:
df_hulk.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
22303,192.168.1.68,57634,192.168.10.124,80,0,0,22303,24,0,0,...,0.0,2,16,3,7,589.083333,524.166667,29.166667,2635.833333,Hulk
22304,192.168.1.68,57636,192.168.10.124,80,0,0,22304,25,0,0,...,0.0,2,16,2,8,527.520000,462.800000,28.800000,2084.560000,Hulk
22305,192.168.1.68,57638,192.168.10.124,80,0,0,22305,26,0,0,...,0.0,2,16,2,9,507.884615,443.346154,28.461538,2002.730769,Hulk
22306,192.168.1.68,57640,192.168.10.124,80,0,0,22306,26,0,0,...,0.0,2,18,3,7,564.076923,498.615385,29.846154,2447.846154,Hulk
22307,192.168.1.68,57642,192.168.10.124,80,0,0,22307,23,0,0,...,0.0,2,15,2,7,567.391304,502.521739,29.043478,2265.304348,Hulk


In [78]:
df_benign_high_latency.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
31830,192.168.2.83,60473,192.168.10.124,80,0,0,31830,1324,0,0,...,0.0,2,1323,2,0,925.622356,869.755287,20.027190,3995.963746,Benign high latency 2
31831,192.168.2.83,60474,192.168.10.124,80,0,0,31831,1328,0,0,...,0.0,2,1327,2,0,924.533886,868.654367,20.045181,4000.838102,Benign high latency 2
31832,192.168.2.83,60476,192.168.10.124,80,0,0,31832,1326,0,0,...,0.0,2,1325,2,0,925.077677,869.204374,20.036199,3998.404977,Benign high latency 2
31833,192.168.2.83,60475,192.168.10.124,80,0,0,31833,1326,0,0,...,0.0,2,1325,2,0,925.077677,869.204374,20.036199,3998.404977,Benign high latency 2
31834,192.168.2.83,60477,192.168.10.124,80,0,0,31834,1323,0,0,...,0.0,2,1322,2,0,925.510204,869.650794,20.018141,3990.497354,Benign high latency 2


In [79]:
df_benign_low_latency.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
0,192.168.2.183,59163,192.168.10.124,80,0,0,0,1367,0,0,...,0.0,2,1366,2,0,898.617410,842.738844,20.096562,3862.054133,Benign low latency 2
1,192.168.2.183,59164,192.168.10.124,80,0,0,1,1609,0,0,...,0.0,2,1607,3,1,780.998135,724.753263,20.723431,3281.287756,Benign low latency 2
2,192.168.2.183,59165,192.168.10.124,80,0,0,2,1603,0,0,...,0.0,2,1602,2,0,783.492202,727.261385,20.711167,3293.467249,Benign low latency 2
3,192.168.2.183,59166,192.168.10.124,80,0,0,3,1355,0,0,...,0.0,2,1354,2,0,905.837638,849.960148,20.079705,3896.256827,Benign low latency 2
4,192.168.2.183,59167,192.168.10.124,80,0,0,4,1381,0,0,...,0.0,2,1380,2,0,890.437364,834.551774,20.121651,3822.902245,Benign low latency 2


### Rough Work

#### Check for nan values

In [17]:
df_slow_http_get_packets.isna().sum()

No.                    0
Time                   0
Delta                  0
Source                 0
Destination            0
Protocol               0
Stream index           0
Length                 0
TCP Segment Len        0
Source Port            0
Header Length          0
Bytes In Flight    90102
Push                   0
Syn                    0
Acknowledgment         0
Fin                    0
Reset                  0
Info                   0
dtype: int64

In [19]:
df_slow_read_packets.isna().sum()

No.                    0
Time                   0
Delta                  0
Source                 0
Destination            0
Protocol               0
Stream index           0
Length                 0
TCP Segment Len        0
Source Port            0
Header Length          0
Bytes In Flight    23013
Push                   0
Syn                    0
Acknowledgment         0
Fin                    0
Reset                  0
Info                   0
dtype: int64

In [20]:
df_slow_post_packets.isna().sum()

No.                    0
Time                   0
Delta                  0
Source                 0
Destination            0
Protocol               0
Stream index           0
Length                 0
TCP Segment Len        0
Source Port            0
Header Length          0
Bytes In Flight    50862
Push                   0
Syn                    0
Acknowledgment         0
Fin                    0
Reset                  0
Info                   0
dtype: int64

In [21]:
df_goldeneye_packets.isna().sum()

No.                    0
Time                   0
Delta                  0
Source                 0
Destination            0
Protocol               0
Stream index           0
Length                 0
TCP Segment Len        0
Source Port            0
Header Length          0
Bytes In Flight    59289
Push                   0
Syn                    0
Acknowledgment         0
Fin                    0
Reset                  0
Info                   0
dtype: int64

In [22]:
df_hulk_packets.isna().sum()

No.                     0
Time                    0
Delta                   0
Source                  0
Destination             0
Protocol                0
Stream index            0
Length                  0
TCP Segment Len         0
Source Port             0
Header Length           0
Bytes In Flight    119110
Push                    0
Syn                     0
Acknowledgment          0
Fin                     0
Reset                   0
Info                    0
dtype: int64

In [23]:
df_benign_high_latency_packets.isna().sum()

No.                    0
Time                   0
Delta                  0
Source                 0
Destination            0
Protocol               0
Stream index           0
Length                 0
TCP Segment Len        0
Source Port            0
Header Length          0
Bytes In Flight    19744
Push                   0
Syn                    0
Acknowledgment         0
Fin                    0
Reset                  0
Info                   0
dtype: int64

In [24]:
df_benign_low_latency_packets.isna().sum()

No.                     0
Time                    0
Delta                   0
Source                  0
Destination             0
Protocol                0
Stream index            0
Length                  0
TCP Segment Len         0
Source Port             0
Header Length           0
Bytes In Flight    107832
Push                    0
Syn                     0
Acknowledgment          0
Fin                     0
Reset                   0
Info                    0
dtype: int64

In [27]:
df = get_full_conversation_df()

In [28]:
df.isna().sum()

Address-A                    0
Port-A                       0
Address-B                    0
Port-B                       0
Packets                      0
Bytes                        0
StreamID                     0
TotalPackets                 0
PerctageFiltered             0
Packets_A-to-B(TX_pkts)      0
Bytes_A-to-B(TX_bytes)       0
Packets_B-to-A(RX_pkts)      0
Bytes_B-to-A(RX_bytes)       0
Relative_Start               0
Duration                     0
Bits/s A-to-B (TX_bits/s)    9
Bits/s B-to-A (RX_bits/s)    9
dtype: int64

In [29]:
len(df)

31878

In [43]:
df_slow_http_get_packets = remove_nan_Bytes_In_Flight(df_slow_http_get_packets)
# df_slow_http_get = 

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Bits/s B-to-A (RX_bits/s),Syn,Acknowledgment,Fin,Reset,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY
Stream index,,,,,,,,,,,,,,,,,,,,,


In [31]:
new_df.columns

Index(['Unnamed: 0', 'Address-A', 'Port-A', 'Address-B', 'Port-B', 'Packets',
       'Bytes', 'StreamID', 'TotalPackets', 'PerctageFiltered',
       'Packets_A-to-B(TX_pkts)', 'Bytes_A-to-B(TX_bytes)',
       'Packets_B-to-A(RX_pkts)', 'Bytes_B-to-A(RX_bytes)', 'Relative_Start',
       'Duration', 'Bits/s A-to-B (TX_bits/s)', 'Bits/s B-to-A (RX_bits/s)',
       'tot_kbps', 'Pktrate', 'Flow Duration', 'Idle Timeout', 'Hard Timeout',
       'Syn', 'Acknowledgment', 'Fin', 'Reset', 'Length', 'TCP Segment Len',
       'Header Length', 'Bytes In Flight', 'CATEGORY'],
      dtype='object')